# DFL benchmark - training
This is a simple benchmark script for DFL.  
It classifies each frame image in the video into 4 classes（'background','challenge','play','throwin'） 
It does not use temporal information, so it may not be competitive on its own for this competition, but it could be used as a feature extractor for more advanced models.

## 画像保存方針
- start 1枚(eventのannotation時間の1フレーム分前の時間)
- event 1枚(1 annotationごとに1枚)
- end 1枚(eventのannotation時間の1フレーム分あとの時間)

In [1]:
!nvidia-smi

Sun Sep 11 16:34:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P8    38W / 350W |    769MiB / 24576MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from IPython.display import Video
import cv2

# setting

In [31]:
DEBUG = False

In [32]:
class CFG:
    train_csv = "/workdir/work/input/train.csv"
    result_dir = f"/workdir/work/output/event4image/"
    video_dir = "/workdir/work/input/train"

In [33]:
# save images around event in result dir
if not os.path.exists(CFG.result_dir):
    os.makedirs(CFG.result_dir)

In [34]:
df = pd.read_csv(CFG.train_csv)
df = df[["video_id", "time", "event"]]
display(df)
video_ids = df["video_id"].unique()
if DEBUG:
    video_ids = ["1606b0e6_0", "ecf251d4_0"]
print(video_ids)

,video_id,time,event
0,1606b0e6_0,200.265822,start
1,1606b0e6_0,201.150000,challenge
2,1606b0e6_0,202.765822,end
3,1606b0e6_0,210.124111,start
4,1606b0e6_0,210.870000,challenge
...,...,...,...
11213,ecf251d4_0,3056.587000,challenge
11214,ecf251d4_0,3058.072895,end
11215,ecf251d4_0,3068.280519,start
11216,ecf251d4_0,3069.547000,throwin


['1606b0e6_0' '1606b0e6_1' '35bd9041_0' '35bd9041_1' '3c993bd2_0'
 '3c993bd2_1' '407c5a9e_1' '4ffd5986_0' '9a97dae4_1' 'cfbe2e94_0'
 'cfbe2e94_1' 'ecf251d4_0']


In [35]:
def make_saved_event_df(save_event_df_, saved_frame_list_, target_, video_id_):
    if len(save_event_df_) == 0:
        save_event_df_ = pd.DataFrame(saved_frame_list_, columns=["frame"])
        save_event_df_["event"] = [target_]*len(saved_frame_list_)
        save_event_df_["video_id"] = [video_id_]*len(saved_frame_list_)
    else:
        tmp_event_df_ = pd.DataFrame(saved_frame_list_, columns=["frame"])
        tmp_event_df_["event"] = [target_]*len(saved_frame_list_)
        tmp_event_df_["video_id"] = [video_id_]*len(saved_frame_list_)
        save_event_df_ = pd.concat([save_event_df_, tmp_event_df_])
    return save_event_df_

In [36]:
def extract_training_images(video_id):
    saved_image_df = pd.DataFrame()
    video_path = f"{CFG.video_dir}/{video_id}.mp4"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"video {video_path} cannot be opened.")
        return
    fps = cap.get(cv2.CAP_PROP_FPS)
    time_interval = 1/fps
        
    df_video = df[df.video_id == video_id]
    if DEBUG:
        df_video = df_video.head(6)
    print(video_id, df_video.shape)

    #crr_statu => background, play, challenge, throwin
    time_and_event = df_video[['time','event']].values
    idx = 0
    saved_frame_list = []
    save_event_df = pd.DataFrame()
    event_start_time = 0
    event_end_time = 0

    while idx < len(time_and_event):
        if DEBUG and idx > 2:
            break

        current_time = time_and_event[idx,0]
        current_event = time_and_event[idx,1]
        if current_event == 'start':
            target = 'background'
            event_start_time = current_time          
            idx += 1
            # save image
            frame_num = int((current_time-time_interval)*fps)
            if not frame_num in saved_frame_list:
                # set frame num and read image
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                successed, frame_image = cap.read()
                if not successed:
                    break
                out_file = f'{CFG.result_dir}/{video_id}_{frame_num:06}.jpg'
                cv2.imwrite(out_file, frame_image)
                saved_frame_list.append(frame_num)
                # saving df
                if len(save_event_df) == 0:
                    save_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                else:
                    tmp_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                    save_event_df = pd.concat([save_event_df, tmp_event_df])
        
        elif current_event == 'end':
            target = 'background'
            idx += 1
            frame_num = int((current_time+time_interval)*fps)
            if not frame_num in saved_frame_list:
                # set frame num and read image
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                successed, frame_image = cap.read()
                if not successed:
                    break
                out_file = f'{CFG.result_dir}/{video_id}_{frame_num:06}.jpg'
                cv2.imwrite(out_file, frame_image)
                saved_frame_list.append(frame_num)
                # saving df
                if len(save_event_df) == 0:
                    save_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                else:
                    tmp_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                    save_event_df = pd.concat([save_event_df, tmp_event_df])

        else:
            target = current_event
            frame_num = int(current_time*fps)
            if not frame_num in saved_frame_list:
                # set frame num and read image
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                successed, frame_image = cap.read()
                if not successed:
                    break
                out_file = f'{CFG.result_dir}/{video_id}_{frame_num:06}.jpg'
                cv2.imwrite(out_file, frame_image)
                saved_frame_list.append(frame_num)
                # saving df
                if len(save_event_df) == 0:
                    save_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                else:
                    tmp_event_df = pd.DataFrame([[video_id, frame_num, (frame_num/fps), target]], columns=["video_id", "frame", "time", "event"])
                    save_event_df = pd.concat([save_event_df, tmp_event_df])
            idx += 1
    
    return save_event_df

In [37]:
%%time

train_images_df = pd.DataFrame()
for video_id in video_ids:            
    video_event_df = extract_training_images(video_id)
    if len(train_images_df) == 0:
        train_images_df = video_event_df
    else:
        train_images_df = pd.concat([train_images_df, video_event_df])
print('done')

1606b0e6_0 (1000, 3)
1606b0e6_1 (1249, 3)
35bd9041_0 (1075, 3)
35bd9041_1 (933, 3)
3c993bd2_0 (1042, 3)
3c993bd2_1 (966, 3)
407c5a9e_1 (858, 3)
4ffd5986_0 (792, 3)
9a97dae4_1 (737, 3)
cfbe2e94_0 (823, 3)
cfbe2e94_1 (763, 3)
ecf251d4_0 (980, 3)
done
CPU times: user 2h 20min 25s, sys: 4min 29s, total: 2h 24min 54s
Wall time: 16min 44s


In [38]:
result_df = train_images_df[["video_id", "frame", "time", "event"]]
result_df.to_csv("/workdir/work/output/saved_train_event4images.csv", index=False)
print(f"{len(result_df)} images are saved.")
print("Completed.")

11182 images are saved.
Completed.
